In [1]:
import os
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score

# get version
print("pandas version",pd.__version__)

fp = "G:\\My Drive\\Darby Work\\XRF fundamentals vs. MVA\\"

pandas version 1.3.2


#### Spectra & metadata

In [2]:
spectra_path = "Z:\\data_pXRF\\MHC_Olympus_spectra.csv"
spectra = pd.read_csv(spectra_path)

metadata_path = "Z:\\data_pXRF\\MHC_Olympus_metadata.csv"
metadata = pd.read_csv(metadata_path)
comps = metadata.drop(metadata.columns[1:8], axis=1)
comps.columns = comps.columns.map(lambda x: x.split()[0])

In [3]:
# select samples with olympus predictions
pred_samples = list(metadata[
    metadata['Olympus-Predicted'] == 'yes'
]['pkey'])

# add wave column
pred_samples.insert(0, 'wave')


pred_spectra = spectra.filter(
    items = pred_samples
)

In [4]:
o1_spectra = pred_spectra.filter(regex=('(_1$)|(wave)'), axis=1)
o2_spectra = pred_spectra.filter(regex=('(_2$)|(wave)'), axis=1)

#### Sensitivity

In [6]:
sens = pd.read_csv(fp+'sensitivities.csv')

In [8]:
# using median for filter 1, mean for filter 2
o1_sens = sens[sens['filter']==1]['median_sens'].iloc[0]
o2_sens = sens[sens['filter']==2]['mean_sens'].iloc[0]

#### Calculate LBDQs from model and instrument sensitivity

In [24]:
def get_lbdq(folder, file_list):
     
    print("LBDQ:")
    
    coeffs = []
    elem_list = []
    filt_list = []
    lob_list = []
    lod_list = []
    loq_list = []
    
    for filter_n in ['O1', 'O2']:
        
        # get sensitivity value
        sensitivity = o1_sens if filter_n == 'O1' else o2_sens
    
        ftype = filter_n + "_coeff"

        # read models
        for file in tqdm(file_list):
            if ftype in file:       
                path = folder + file
                data = pd.read_csv(path, skiprows = [0])

                # convert to dataframe
                data = data.T

                # adapt to different element naming b/w datasets
                data.columns = data.iloc[0].map(lambda x: x.split()[0])
                data = data.drop(data.index[0])
                element = data.columns[0]
                
                # populate lists
                elem_list.append(element)
                filt_list.append(filter_n)

                # calculate regression vectors
                vector = pow(data, 2).sum().pow(.5)  #square root of sum of squares
                
                # calculate values
                factors = {
                    'LOB' : 1.645,
                    'LOD' : 3.3,
                    'LOQ' : 10
                }

                lob_list.append(factors['LOB'] * sensitivity * vector[0])
                lod_list.append(factors['LOD'] * sensitivity * vector[0])
                loq_list.append(factors['LOQ'] * sensitivity * vector[0])

    # make dataframe
    df = pd.DataFrame({
        'element' : elem_list,
        'filter' : filt_list,
        'LOB' : lob_list,
        'LOD' : lod_list,
        'LOQ' : loq_list
    })
    
    # change col formats
    cols = df.columns.drop(['element', 'filter'])
    df[cols] = df[cols].apply(pd.to_numeric)
    
    return df

#### Calculate RMSEPs

In [20]:
def get_rmsep(folder, file_list, lbdq):
    
    print("RMSEP:")
    
    elem_list = []
    filt_list = []
    avg_list = []
    rmsep_list = []
    r2_list = []
    
    for filter_n in ['O1', 'O2']:
    
        ftype = filter_n + "_test"

        for file in tqdm(file_list):
            if ftype in file:       
                path = (folder + file)
                data = pd.read_csv(path)

                # get element
                element = data.columns[1].split()[0]
                elem_list.append(element)
                filt_list.append(filter_n)

                # format columns
                data.columns = ['pkey', 'Actual', 'Pred']
                data = data.drop([0])
                data.Pred = data.Pred.astype(float)  

                # remove predictions above 100 for majors
                if len(element) > 2:
                    data = data[data.Pred < 100]

                # remove all predictions below 0
                data = data[data.Pred > 0].reset_index(drop=True).sort_index(axis=1)

                # order columns
                data = data[['pkey', 'Actual', 'Pred']].drop_duplicates(subset = 'pkey').sort_values(by='pkey').reset_index(drop=True)

                # subselect relevant reference values
                ref = lbdq[(lbdq['element'].astype(str) == element) &
                           (lbdq['filter'].astype(str) == filter_n)].reset_index(drop=True)

                # add in Actual concentrations
                temp_comps = comps[comps.pkey.isin(data.pkey)].reset_index(drop=True)                   
                data['Actual'] = temp_comps[temp_comps['pkey'] == data['pkey']][element]

                # remove NaN Actual values....which idk why they'd be there
                data = data.dropna()

                # calculate values
                loq = ref['LOQ'].iloc[0]
                # select just predictions above the LOQ
                data = data[data.Pred > loq].reset_index(drop=True)
                # get average concentration
                avg = data['Actual'].mean()
                avg_list.append(avg)
                # get R2
                if len(data) > 1:
                    r2 = r2_score(data.Actual, data.Pred)
                    r2_list.append(r2)
                else: r2_list.append('Not enough test samples above LOQ')
                # get RMSE-P
                data['sqerror'] = (data.Actual - data.Pred).pow(2)
                rmsep = data['sqerror'].mean() ** 0.5
                rmsep_list.append(rmsep)

    
    df = pd.DataFrame({
        "element" : elem_list,
        "filter" : filt_list,
        "avg_comp" : avg_list,
        "RMSEP" : rmsep_list,
        "R2" : r2_list,
    })
    
    
    return df

In [30]:
def get_results(regression, n_range):
    
    print('Calculating for', regression, n_range)

    folder = fp+"\\models\\"+regression+"\\"+n_range+"\\"
    file_list = os.listdir(folder)
    if len(file_list) == 0:
        print("\tNo files")
        return

    # calculate lbdq
    lbdq = get_lbdq(folder, file_list)
    # calculate rmsep with lbdq results
    rmsep = get_rmsep(folder, file_list, lbdq)
    # merge results
    df = pd.merge(lbdq, rmsep, how='outer', on=['element', 'filter'])
    df.insert(loc=2, column='num_range', value=n_range)
    df.insert(loc=2, column='regression', value=regression)
    # return full results
    return df 

### Conduct calculations

In [33]:
# get results for all data
full_table = pd.DataFrame()

regressions = ['PLS', 'lasso']
regions = ['0-750', '250-1000', 'all']

for regression in regressions:
    for region in regions:
        results = get_results(regression, region)
        full_table = pd.concat([full_table, results], ignore_index=True)
        
full_table.reindex(columns=['element',
                            'filter',
                            'regression',
                            'num_range',
                            'avg_comp',
                            'LOB',
                            'LOD',
                            'LOQ',
                            'RMSEP',
                            'R2'])

full_table

Calculating for PLS 0-750
LBDQ:


  0%|          | 0/240 [00:00<?, ?it/s]

  0%|          | 0/240 [00:00<?, ?it/s]

RMSEP:


  0%|          | 0/240 [00:00<?, ?it/s]

...

  0%|          | 0/177 [00:00<?, ?it/s]

RMSEP:


  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

,element,filter,regression,num_range,LOB,LOD,LOQ,avg_comp,RMSEP,R2
0,Al2O3,O1,PLS,0-750,0.079006,0.158492,0.480279,13.802851,2.953625,0.292765
1,CaO,O1,PLS,0-750,0.081649,0.163794,0.496346,6.539167,2.426925,0.708863
2,Fe2O3,O1,PLS,0-750,0.008947,0.017949,0.054392,8.092070,1.314764,0.924568
3,MgO,O1,PLS,0-750,0.006960,0.013963,0.042312,6.258273,5.231678,0.328343
4,MnO,O1,PLS,0-750,0.008493,0.017038,0.051632,0.165387,0.058478,0.932526
...,...,...,...,...,...,...,...,...,...,...
355,V,O2,lasso,all,41.458137,83.168300,252.025150,NaN,NaN,NaN
356,W,O2,lasso,all,2.326984,4.668114,14.145799,NaN,NaN,NaN
357,Y,O2,lasso,all,1.714465,3.439352,10.422280,NaN,NaN,NaN
358,Zn,O2,lasso,all,11.725278,23.521834,71.278283,NaN,NaN,NaN


#### Export

In [32]:
full_epath = fp + "full_results_table.csv"
full_table.to_csv(full_epath, index=False)